In [13]:
import os
import torch
import pandas as pd
import scanpy as sc
from sklearn import metrics
import multiprocessing as mp
from GraphST import GraphST
import numpy as np
import anndata as ad
import scanpy as sc
import os
import skmisc
from sklearn.neighbors import NearestNeighbors

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
cluster = "/fs/cbsuvlaminck2"
workdir = "/workdir/bmg224/harvard_dental/manuscript/code"
os.chdir(cluster + workdir)


In [4]:
os.getcwd()

'/fs/cbsuvlaminck2/workdir/bmg224/harvard_dental/manuscript/code'

In [5]:
out_dir = "../outputs/segmentation_2024_03_07/{date}/{date}_{sn}"
out_fmt_classif = out_dir + "/classif"
centroid_sciname_fmt = out_fmt_classif + "/{date}_{sn}_centroid_sciname.csv"

In [6]:
date = "2023_02_08"
sn = "hsdm_group_1_sample_12_fov_01"

centroid_sciname_fn = centroid_sciname_fmt.format(date=date, sn=sn)
centroid_sciname = pd.read_csv(centroid_sciname_fn)
coords = np.array([eval(c) for c in centroid_sciname["coord"].values])
scinames = centroid_sciname["sciname"].values
scn_unq = np.unique(scinames)

## Build adata object

Get feature matrix as cell identity

In [7]:
X = []
for scn in scinames:
    x = np.zeros(len(scn_unq))
    idx = np.where(scn == scn_unq)
    x[idx] = 1
    X.append(x)
X = np.array(X)
adata = ad.AnnData(X)
adata.var_names = scn_unq

In [8]:
adata.obsm['spatial'] = coords

In [9]:
adata

AnnData object with n_obs × n_vars = 69665 × 8
    obsm: 'spatial'

Get feature matrix as neighbors

## Run GraphST

In [10]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [11]:
os.environ['R_HOME'] = '~/miniconda3/envs/GraphST/lib/R/bin/R'

In [12]:
model = GraphST.GraphST(adata, device=device)

ValueError: b'There are other near singularities as well. 0.10808'